# Exploring eBay Car Sales

We will clean and explore this eBay car sales dataset with 50K data points. We are interested in finding out if for the top 10 car brand listings on the website there exists a correlation between the average price of the car and the average mileage of the car.

In [1]:
import pandas as pd

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

We can see that the dataset has 20 columns where the column names follow the camelcase and we would like to use snakecase. So we will go ahead and change the column names. 

In [2]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [3]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_of_pictures', 'postal_code',
       'last_seen']

In [4]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
date_crawled          50000 non-null object
name                  50000 non-null object
seller                50000 non-null object
offer_type            50000 non-null object
price                 50000 non-null object
abtest                50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gearbox               47320 non-null object
power_ps              50000 non-null int64
model                 47242 non-null object
odometer              50000 non-null object
registration_month    50000 non-null int64
fuel_type             45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
num_of_pictures       50000 non-null int64
postal_code           50000 non-null int64
last_seen             50000 non-null object
dtypes: int64(5)

Now we have a nice and clean snakecase column names. We notice that the price and odometer columns are not numeric. We need them to be numeric so that we can compute averages and basically perform math on these numbers. We will also change the odometer name to odometer_km so that we know we are working with kilometers.

In [5]:
autos["price"] = autos["price"].str.replace("$", "").str.replace(",", "").astype(int)
autos["odometer"] = autos["odometer"].str.replace("km", "").str.replace(",", "").astype(int)
autos = autos.rename({"odometer": "odometer_km"}, axis = 1)
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Let's check if there are any columns that we can remove because the are more or less consistent or we are not really interested in them.

In [6]:
print(autos["seller"].unique())
print(autos["offer_type"].unique())
print(autos["num_of_pictures"].unique())
autos = autos.drop(columns = ["seller", "offer_type", "num_of_pictures"])

['privat' 'gewerblich']
['Angebot' 'Gesuch']
[0]


Now that we got rid of some columns that were not providing interesting information lets move on to outliers in the dataset.

In [7]:
autos.describe()

,price,registration_year,power_ps,odometer_km,registration_month,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,9.840044e+03,2005.073280,116.355920,125732.700000,5.723360,50813.627300
std,4.811044e+05,105.712813,209.216627,40042.211706,3.711984,25779.747957
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,1.100000e+03,1999.000000,70.000000,125000.000000,3.000000,30451.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,49577.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,71540.000000
max,1.000000e+08,9999.000000,17700.000000,150000.000000,12.000000,99998.000000


From the distribution we can see that price, registration year and odometer have some outliers we will check all of these columns and get rid of the outliers.

In [8]:
print("Top 5 least expensive cars:\n", autos["price"].value_counts().sort_index().head(),"\n")
print("Top 5 most expensive cars:\n", autos["price"].value_counts().sort_index(ascending = False).head())

Top 5 least expensive cars:
 0    1421
1     156
2       3
3       1
5       2
Name: price, dtype: int64 

Top 5 most expensive cars:
 99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
Name: price, dtype: int64


In [9]:
#Only makes sense to have the price of cars between 100 and 100K
autos = autos[autos["price"].between(100,100000)] 

In [10]:
print("Top 5 least milage:\n", autos["odometer_km"].value_counts().sort_index().head(),"\n")
print("Top 5 most milage:\n", autos["odometer_km"].value_counts().sort_index(ascending = False).head())

Top 5 least milage:
 5000     751
10000    242
20000    749
30000    771
40000    813
Name: odometer_km, dtype: int64 

Top 5 most milage:
 150000    31209
125000     5035
100000     2099
90000      1733
80000      1411
Name: odometer_km, dtype: int64


In [11]:
# Only makes sense to have a milage of 10k to 150k 
autos = autos[autos["odometer_km"].between(10000,150000)]

In [12]:
print("Top 5 oldest cars:\n", autos["registration_year"].value_counts().sort_index().head(),"\n")
print("Top 5 newest cars:\n", autos["registration_year"].value_counts().sort_index(ascending = False).head())

Top 5 oldest cars:
 1910    1
1931    1
1934    1
1937    2
1941    2
Name: registration_year, dtype: int64 

Top 5 newest cars:
 9999    3
9000    1
8888    1
6200    1
5911    1
Name: registration_year, dtype: int64


In [13]:
# Only makes sense to have a registration year from 1900 till 2019
autos = autos[autos["registration_year"].between(1900,2019)]
autos.describe()

,price,registration_year,power_ps,odometer_km,registration_month,postal_code
count,47427.000000,47427.000000,47427.000000,47427.000000,47427.000000,47427.000000
mean,5777.421216,2003.533156,117.791553,127919.644928,5.826976,51045.792861
std,7435.396984,7.327546,201.551156,36675.088739,3.666954,25743.305971
min,100.000000,1910.000000,0.000000,10000.000000,0.000000,1067.000000
25%,1250.000000,1999.000000,75.000000,125000.000000,3.000000,30826.000000
50%,3012.000000,2004.000000,109.000000,150000.000000,6.000000,49740.000000
75%,7498.000000,2008.000000,150.000000,150000.000000,9.000000,71706.000000
max,99900.000000,2019.000000,17700.000000,150000.000000,12.000000,99998.000000


Now the distributions make more sense after taking the outliers out. The data is cleaned let's investigate what is the mean price of the top 10 car brands on sale.

In [14]:
top_brands = autos["brand"].value_counts().head(10).index

brand_mean_prices = {}

for brand in top_brands:
    current_brand = autos[autos["brand"]==brand]
    mean_price = current_brand["price"].mean()
    brand_mean_prices[brand] = round(mean_price,1)

bmp_series = pd.Series(brand_mean_prices)
bmp_df = pd.DataFrame(bmp_series, columns=['mean_price'])
print(bmp_df)

               mean_price
audi               9194.5
bmw                8194.7
fiat               2767.4
ford               3648.9
mercedes_benz      8443.4
opel               2949.4
peugeot            3084.9
renault            2414.8
seat               4309.9
volkswagen         5374.4


Let's see if there is a correlation between mean milage of these brands and their mean price.

In [15]:
brand_mean_mileage = {}

for brand in top_brands:
    current_brand = autos[autos["brand"]==brand]
    mean_mileage = current_brand["odometer_km"].mean()
    brand_mean_mileage[brand] = round(mean_mileage,1)

    
bmm_series = pd.Series(brand_mean_mileage)
bmp_df["mean_mileage"] = bmm_series
print(bmp_df)

               mean_price  mean_mileage
audi               9194.5      130543.0
bmw                8194.7      134716.4
fiat               2767.4      120073.6
ford               3648.9      126299.2
mercedes_benz      8443.4      132687.8
opel               2949.4      131894.8
peugeot            3084.9      128271.8
renault            2414.8      130603.2
seat               4309.9      123579.4
volkswagen         5374.4      130858.2


Interestingly the mean mileage for the top 10 brands is roughly the same this implies that at least for these brands the price is independent of the milage.